In [5]:
import tensorflow as tf
import numpy as np
import glob
import os
import sys
import pandas as pd
import image_aug as im
import cv2 as cv

In [6]:
imagedir = '/home/abake116/Fish/test_stg1/'
modelFullPath = '/home/abake116/Fish/models/inc_retrain_full/output_graph.pb'
labelsFullPath = '/home/abake116/Fish/models/inc_retrain_full/output_labels.txt'

In [7]:
images = glob.glob(imagedir + '*.jpg')

In [8]:
df.head()

NameError: name 'df' is not defined

In [9]:
headers = ['image','alb','bet','dol','lag','nof','other','shark','yft']
df = pd.DataFrame(np.nan, index=(range(len(images))),columns=headers)

In [10]:
def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [31]:
def run_inference_on_image(img,num_aug):
    filename = os.path.basename(img)
    answer = {'image':filename,'alb':0,'bet':0,'dol':0,'lag':0,'nof':0,'other':0,'shark':0,'yft':0}
    #if not tf.gfile.Exists(imagePath):
    #    tf.logging.fatal('File does not exist %s', imagePath)
    #    return answer

    image_data = tf.gfile.FastGFile(img, 'rb').read()
    print tf.shape(image_data)

    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:

        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        print tf.shape(image_data)
        for i in range(num_aug):
            
            image_data = tf.image.flip_up_down(image_data)
            image_data = tf.image.resize_images(image_data, [299,299,3], 
                                                method=ResizeMethod.NEAREST_NEIGHBOR, 
                                                align_corners=False)
            
            if i == 0:
                predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})
            else:
                predictions += sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})
                
            
        predictions = np.squeeze(predictions)
        
        predictions /= num_aug

        top_k = predictions.argsort()[-8:][::-1]  # Getting top 5 predictions
        f = open(labelsFullPath, 'rb')
        lines = f.readlines()
        labels = [str(w).replace("\n", "") for w in lines]
        for node_id in top_k:
            human_string = labels[node_id]
            score = predictions[node_id]
            answer[human_string] = score
        return answer

In [32]:
#Fix this is batch predictions
if __name__ == '__main__':
    
    for i,img in enumerate(images):
        with tf.Graph().as_default():
            answer = run_inference_on_image(img,5)
            print i
            for x in answer:
                df[x].iloc[i] = answer[x]

Tensor("Shape:0", shape=(0,), dtype=int32)
Tensor("Shape_1:0", shape=(0,), dtype=int32)


ValueError: 'image' must be three-dimensional.

In [53]:
df.to_csv('retrain_full_001_10000_bright5.csv',index=False)